## BEFORE THE WEB

O1sSgDlqAGPWi0pvinvP.grehack.fr:70



Notes:

- Thử intercept với burpsuite nhưng không thành công. Do đó luật điểm suy nghĩ before the web = proxy là **không hợp lý**.
- port 70 = gopher
- Thử sử dụng: https://gopherproxy.meulie.net/o1ssgdlqagpwi0pvinvp.grehack.fr Thấy được các danh sách files trên server
- Có 1 cái CWE path traversal attacks.


### Phân tích

- Theo wiki thì trước khi web thống trị thì gopher là một trong những giao thức được sử dụng để truyền đạt thông tin. Do đó gopher có trước web.
- port của gopher là port 70.

Do đó 2 luận điểm trên là *hợp lý*.

### OK. Bây giờ sử dụng gopher và tấn công path traversal lấy password.

```
# CWE-22: Improper Limitation of a Pathname to a Restricted Directory ('Path Traversal')
## Description Summary
The software uses external input to construct a pathname that is intended to identify a file or directory that is located underneath a restricted parent directory, but the software does not properly neutralize special elements within the pathname that can cause the pathname to resolve to a location that is outside of the restricted directory.

## Extended Description
Many file operations are intended to take place within a restricted directory. By using special elements such as ".." and "/" separators, attackers can escape outside of the restricted location to access files or directories that are elsewhere on the system. One of the most common special elements is the "../" sequence, which in most modern operating systems is interpreted as the parent directory of the current location. This is referred to as relative path traversal. Path traversal also covers the use of absolute pathnames such as "/usr/local/bin", which may also be useful in accessing unexpected files. This is referred to as absolute path traversal.

In many programming languages, the injection of a null byte (the 0 or NUL) may allow an attacker to truncate a generated filename to widen the scope of attack. For example, the software may add ".txt" to any pathname, thus limiting the attacker to text files, but a null injection may effectively remove this restriction.
```


Trong một số programming language, inject null byte có thể cho phép attacker cắt một file name để mở rộng phạm vi tấn công.
Ví dụ phần mềm có phần mở rộng là txt trong pathnam, do đó dùng phần mở rộng giới hạn lại chỉ là text file.
Nhưng kỹ thuật null byte injections có thể qua mặt được restriction này.

### NULL byte

[Tham khảo](https://www.owasp.org/index.php/Path_Traversal)

Trong một số hệ điều hành, null byte là %00. Xét ví dụ:

`?file=secret.doc%00.pdf`

Thì hệ điều hành sẽ đọc file secret.doc trong khi đó web server như PHP hay java sẽ đọc dạng .pdf vì %00 đã bị terminated.


Do proxy online không hoạt động. Tôi nghi ngờ là do giới hạn của web service khi sử dụng dịch vụ online.

Là do: https://gopherproxy.meulie.net/o1ssgdlqagpwi0pvinvp.grehack.fr/0/CWE-22.txt

Khi tôi muốn yêu cầu request là: https://gopherproxy.meulie.net/o1ssgdlqagpwi0pvinvp.grehack.fr/0/../../../etc/password%00.txt thì server phản hồi lại là `https://gopherproxy.meulie.net/etc/passwd%00.txt` dẫn tới lỗi HTTP error 400.

Nên tôi quyết định sẽ tiến hành chạy một client mà không cần dịch vụ này.

Request gửi là: O1sSgDlqAGPWi0pvinvP.grehack.fr:70

### Thử sử dụng curl

Theo trang [này](https://curl.haxx.se/docs/manpage.html) thì curl hỗ trợ gopher. Test bằng lệnh

`curl gopher://o1ssgdlqagpwi0pvinvp.grehack.fr/0/CWE-22.txt `

Thấy ok. Response gồm 4 cột như sau:

```
0CWE-22.txt                 /CWE-22.txt                 O1sSgDlqAGPWi0pvinvP.grehack.fr 70
IGH16_logo.png              /GH16_logo.png              O1sSgDlqAGPWi0pvinvP.grehack.fr 70
IGH16_logo_black.png        /GH16_logo_black.png        O1sSgDlqAGPWi0pvinvP.grehack.fr 70
IGH16_logo_txt.png          /GH16_logo_txt.png          O1sSgDlqAGPWi0pvinvP.grehack.fr 70
IGH16_logo_txt_black.png    /GH16_logo_txt_black.png    O1sSgDlqAGPWi0pvinvP.grehack.fr 70

1camelid                    /camelid                    O1sSgDlqAGPWi0pvinvP.grehack.fr 70
1passwd(5)_history          /passwd(5)_history          O1sSgDlqAGPWi0pvinvP.grehack.fr 70
1spec                       /spec                       O1sSgDlqAGPWi0pvinvP.grehack.fr 70

```

Các số 0,1 theo trang gopher online thì là directories.

- Các file logo không quan trọng vì nó là gopher image.
- passwd_history nêu thông tin các tập tin password của hệ điều hành FreeBSD, Ubuntu.
- camelid theo google là những con lạc đà nhưng tại sao **kết nối hay bị reset**.? Có lẽ do hình có kích thước lớn.
- cái spec chứa 2 RFC: rfc1436.txt (**The Internet Gopher Protocol**) và rfc4266.txt (**The gopher URI Scheme**)

Có lẽ dụng ý của tác giả là hiểu nguyên tắc protocol và URI scheme. Sau đó tấn công path traversal attacks bằng cách modify URI  bằng null byte trên URI scheme các file có đuôi .txt. Từ đó lấy được password từ tập tin /etc/passwd trong OS.

**Các bước**:

- Tạo một đường dẫn độc hại truy cập tới /etc/passwd
  - Trong đó bao gồm phần mở rộng .txt và có null byte để đánh lừa shell. /etc/passwd%00.txt
- Gửi request tới server và xem response.

password files: 

- /etc/passwd
- /etc/shadow
- /etc/passwd- Backup

FreeBSD:

- /etc/pwd.db
- /etc/master.passwd
- /etc/spwd.db

**Test:**

Khi truy cập đường dẫn `camelid` sẽ hiện ra danh sách các hình có con lạc đà:

`curl gopher://o1ssgdlqagpwi0pvinvp.grehack.fr/1/camelid`

Thử inject null byte, kết quả trả về cùng một danh sách, chứng tỏ web server này có lỗ hổng này.

`curl gopher://o1ssgdlqagpwi0pvinvp.grehack.fr/0/../1/camelid%00spec/rfc4266.txt`


Vậy soạn các url:
```
curl gopher://o1ssgdlqagpwi0pvinvp.grehack.fr/0/../../../../etc/passwd%00spec/rfc4266.txt
curl gopher://o1ssgdlqagpwi0pvinvp.grehack.fr/0/../../../../etc/shadow%00spec/rfc4266.txt
curl gopher://o1ssgdlqagpwi0pvinvp.grehack.fr/0/../../../../etc/pwd.db%00spec/rfc4266.txt
curl gopher://o1ssgdlqagpwi0pvinvp.grehack.fr/0/../../../../etc/master.passwd%00spec/rfc4266.txt
curl gopher://o1ssgdlqagpwi0pvinvp.grehack.fr/0/../../../../etc/spwd.db%00spec/rfc4266.txt
```

Nhưng không cái nào được.

**Solution 1**:

Chắc do sai syntax: `gopher://<host>:<port>/<gopher-path>`

`<gopher-path>` là một trong 3 loại sau:

```xml
     <gophertype><selector> 
     <gophertype><selector>%09<search> 
     <gophertype><selector>%09<search>%09<gopher+_string>
```


The entire `<gopher-path>` may also be empty, in which case the delimiting "/" is also optional and the `<gophertype>` defaults to "1".

`<gophertype>` is a single-character field to denote the Gopher type of the resource to which the URL refers. Ví dụ quy ước loại resource mà URL muốn chỉ định để đáp ứng cho client. Chẳng hạn file, thư mục.

```
  0   Item is a file 
  1   Item is a directory 
  2   Item is a CSO phone-book server 
  3   Error 
  4   Item is a BinHexed Macintosh file. 
  5   Item is DOS binary archive of some sort. 
      Client must read until the TCP connection closes.  Beware. 
  6   Item is a UNIX uuencoded file. 
  7   Item is an Index-Search server. 
  8   Item points to a text-based telnet session. 
  9   Item is a binary file! 
```

`<selector>` is the Gopher selector string.  In the Gopher protocol, Gopher selector strings are a sequence of octets that may contain any octets except 09 hexadecimal (US-ASCII HT or tab), 0A hexadecimal (US-ASCII character LF), and 0D (US-ASCII character CR).